<a href="https://colab.research.google.com/github/sdhilip200/Content-Based-Recommendation---Good-Reads-data/blob/master/Recommendation_Word2Vec.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [62]:
# Importing necessary libraries
import pandas as pd
import numpy as np
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from sklearn.metrics.pairwise import linear_kernel
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.tokenize import RegexpTokenizer
import re
import string
import random
from PIL import Image
import requests
from io import BytesIO
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.metrics.pairwise import cosine_similarity
from gensim.models import Word2Vec
from gensim.models.phrases import Phrases, Phraser
from matplotlib import pyplot
from gensim.models import KeyedVectors

from sentence_transformers import SentenceTransformer, util
import torch

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\maria\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [63]:
# Reading the data
df = pd.read_csv("data.csv")


In [64]:
df.head(5)

,Unnamed: 0.1,Desc,Unnamed: 0,author,genre,image_link,rating,title
0,0,We know that power is shifting: From West to E...,0.0,Moisés Naím,Business,https://i.gr-assets.com/images/S/compressed.ph...,3.63,The End of Power: From Boardrooms to Battlefie...
1,1,Following the success of The Accidental Billio...,1.0,Blake J. Harris,Business,https://i.gr-assets.com/images/S/compressed.ph...,3.94,"Console Wars: Sega, Nintendo, and the Battle t..."
2,2,How to tap the power of social software and ne...,2.0,Chris Brogan,Business,https://i.gr-assets.com/images/S/compressed.ph...,3.78,Trust Agents: Using the Web to Build Influence...
3,3,William J. Bernstein is an American financial ...,3.0,William J. Bernstein,Business,https://i.gr-assets.com/images/S/compressed.ph...,4.20,The Four Pillars of Investing
4,4,Amazing book. And I joined Steve Jobs and many...,4.0,Akio Morita,Business,https://i.gr-assets.com/images/S/compressed.ph...,4.05,Made in Japan: Akio Morita and Sony


In [69]:
# Get list of strings as input
corpus = df['Desc'].to_list()
titles = df['title'].to_list()

In [70]:
catalogue = list(zip(corpus, titles))

# SBERT embeddings

In [77]:
embedder = SentenceTransformer('all-MiniLM-L6-v2')

corpus_embeddings = embedder.encode(corpus, convert_to_tensor=True) #don't run again unless needed!

In [78]:
# Query sentences:
query = ['A married woman has an affair.']


In [79]:
# Recommending the Top K similar books

def recommendations(query, top_k = 3):
    '''
    Input: query as a single-item list
    Finds k nearest descriptions
    Output: titles for k nearest descriptions
    
    '''
    #Compute embeddings for the query
    query_embeddings = embedder.encode(query, convert_to_tensor=True)
    
    #Compute cosine-similarities for the query with each description
    cosine_scores = util.cos_sim(query_embeddings, corpus_embeddings)
    scores = cosine_scores[0].tolist() #get a list of scores

    #Find the pairs with the highest cosine similarity scores
    pairs = [{'index': [i], 'score': scores[i]} for i in range(len(scores)-1)]

    #Sort scores in decreasing order
    pairs = sorted(pairs, key=lambda x: x['score'], reverse=True)

    for pair in pairs[0:top_k+1]:
        i = pair['index'][0]
        print(i)
        print("Title: {} \nDescription: {}\n Score: {:.4f}".format(catalogue[i][1], catalogue[i][0], pair['score']))

## Run the function

In [80]:
recommendations(query) # PoC, now printing descriptions instead of titles

2348
Title: The Wife Between Us 
Description: When you read this book, you will make many assumptions.You will assume you are reading about a jealous ex-wife.You will assume she is obsessed with her replacement – a beautiful, younger woman who is about to marry the man they both love. You will assume you know the anatomy of this tangled love triangle.Assume nothing. Twisted and deliciously chilling, Greer Hendricks and Sarah Pekkanen's The Wife Between Us exposes the secret complexities of an enviable marriage - and the dangerous truths we ignore in the name of love.Read between the lies. 
 Score: 0.4587
2061
Title: The Jane Austen Book Club 
Description: The Extraordinary New York Times Bestseller  In California's central valley, five women and one man join to discuss Jane Austen's novels. Over the six months they get together, marriages are tested, affairs begin, unsuitable arrangements become suitable, and love happens. With her eye for the frailties of human behavior and her ear fo